# Test de match des produits navigo / toflit 
(tests faits avec 2 classifications toflit : orthographic normalization et simplification ==> résultats pour les 2 tests à la fin)

**Procédure :**
- créer des listes de produits de plus en plus proches typographiquement [étapes 1 à 3]
- pour les noms de produits qui ne trouvent pas de match à la fin, on racinise les nomes de produits (stemming) et on utilise des fonctions de similarité pour proposer des termes similaires (candidats au match) [étapes 4 & 5]

0. definition de fonctions 
1. extraction des produits 
2. modifications successives des noms de produits 
3. test des correspondances entre noms de produits nettoyés => obtention de matchs évidents
4. racinisation pour les produits qui n'ont toujours pas trouvé de match
5. fonction de similarité jaccard pour proposer des matchs moins évidents

### bootstraping du client

In [1]:
import json
import csv

# import the lib client
from lib.client import Api
# instantiate the lib client
client = Api()

## 0. definition de fonctions (intersection et différence entre listes)

In [3]:
# intersection pour les listes (complexité en O(n))
def hybrid_intersection(lst1, lst2):  
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return lst3 

# différence entre deux listes
def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif

## 1. extraction des produits

In [4]:
transfo_names_products_navigo = []
with open('dumps/20210208 - cargo dir La Rochelle 08-02-2021.xlsx - id et standardized FR et GB.csv', newline='') as csvfile:
    csv_file = csv.reader(csvfile, quotechar='|')
    k = 0 # compteur pour savoir à quelle ligner on en est
    for row in csv_file:
        k += 1
        if k == 1: 
            continue # on ne veut pas mettre le nom des colonnes dans les produits
        if k == 126:
            break # les noms de produits à partir de la ligne 125 ne sont pas à prendre en compte
        transfo_names_products_navigo.append({'navigo_product_id' : row[0], 'name_navigo' : row[1],})
print ("Nombre de classifications navigo :", len(transfo_names_products_navigo))
# print("toflit orthographic objects:",transfo_names_products_navigo)

result1 = client.toflit.get_classification_search("product_orthographic")
result2 = client.toflit.get_classification_search("product_simplification")

transfo_names_products_toflit_orthographic = []
for s in result1:         
    transfo_names_products_toflit_orthographic.append({'name_toflit_orthographic' : s["name"]})
# print("toflit orthographic objects:",transfo_names_products_toflit_orthographic[0:10])
    
transfo_names_products_toflit_simplification = []
for s in result2:          
    transfo_names_products_toflit_simplification.append({'name_toflit_simplification' : s["name"]}) 
# print("toflit simplification objects:",transfo_names_products_toflit_simplification[0:30])

Nombre de classifications navigo : 124
Nombre de classifications trouvées :  28353
Nombre de classifications trouvées :  20868


## 2. modifications successives des noms de produits enregistrées dans des dictionnaires, et dans des listes

In [5]:
import unidecode, fog
from fog.key import fingerprint, create_fingerprint
f = create_fingerprint(stopwords=['de','du','des','d\'', 'd','en','à', 'a', 'au','le','la','les','l\'', 'l', 'et', 'pour', 'un', 'une']) # au niveau des apostrophes : l' et d' ne fonctionnent pas

navigo_fingerprinted = []
for i in transfo_names_products_navigo:
    i['lowercase_name'] = i['name_navigo'].lower()
    i['typographic_cleaned_name'] = unidecode.unidecode(i['lowercase_name'])
    i['special_charachters_cleaned_name'] = i['typographic_cleaned_name'].replace(",","").replace(";","").replace("[","").replace("]","").replace("-"," ").replace("'"," ")
    i['stop_words_cleaned_name'] = f(fingerprint(i['special_charachters_cleaned_name']))
    navigo_fingerprinted.append(i['stop_words_cleaned_name'])
# print("navigo objects:",transfo_names_products_navigo)

simplification_fingerprinted = []
for i in transfo_names_products_toflit_simplification:
    i['lowercase_name'] = i['name_toflit_simplification'].lower()
    i['typographic_cleaned_name'] = unidecode.unidecode(i['lowercase_name'])
    i['special_charachters_cleaned_name'] = i['typographic_cleaned_name'].replace(",","").replace(";","").replace("[","").replace("]","").replace("-"," ").replace("'"," ")
    i['stop_words_cleaned_name'] = f(fingerprint(i['special_charachters_cleaned_name']))
    simplification_fingerprinted.append(i['stop_words_cleaned_name'])
# print("toflit simplification objects:",transfo_names_products_toflit_simplification[0:30])

orthographic_fingerprinted = []
for i in transfo_names_products_toflit_orthographic:
    i['lowercase_name'] = i['name_toflit_orthographic'].lower()
    i['typographic_cleaned_name'] = unidecode.unidecode(i['lowercase_name'])
    i['special_charachters_cleaned_name'] = i['typographic_cleaned_name'].replace(",","").replace(";","").replace("[","").replace("]","").replace("-"," ").replace("'"," ")
    i['stop_words_cleaned_name'] = f(fingerprint(i['special_charachters_cleaned_name']))
    orthographic_fingerprinted.append(i['stop_words_cleaned_name'])
# print("toflit orthographic objects:",transfo_names_products_toflit_orthographic[0:30])

## 3. test des correspondances entre noms de produits nettoyés
à ce stade les noms de produits sont tous en minuscule, standardisés typographiquement, sans caractères spéciaux et sans stop words

In [6]:
print("\n ********************** Correspondance noms de produits néttoyés **********************")
               
common_navigo_ortho = hybrid_intersection(navigo_fingerprinted, orthographic_fingerprinted)
common_navigo_simpli = hybrid_intersection(navigo_fingerprinted, simplification_fingerprinted)
print("elements communs navigo / toflit orthographic :", len(common_navigo_ortho))
print("elements communs navigo / toflit simplification :", len(common_navigo_simpli))

# Visualisation de ce qui reste à aligner (on décide de procéder avec la convention orthographic simplification)
a_aligner_simplification = Diff(navigo_fingerprinted, common_navigo_simpli)
a_aligner_orthographic = Diff(navigo_fingerprinted, common_navigo_ortho)

# print("\nnavigo / toflit simplification ; reste à aligner à la main ", len(a_aligner_simplification), " produits : ", a_aligner_simplification)
print("\nnavigo / toflit orthographic ; reste à aligner à la main ", len(a_aligner_orthographic), " produits : ", a_aligner_orthographic)


 ********************** Correspondance noms de produits néttoyés **********************
elements communs navigo / toflit orthographic : 95
elements communs navigo / toflit simplification : 93

navigo / toflit orthographic ; reste à aligner à la main  29  produits :  ['soude', 'biscuits', 'ail', 'voiles', 'graines lin', 'bois merrain', 'marchandises naufrage', 'bretagne pressees sardines', 'grain', 'fourrage', 'pierre taille', 'cordages vieux', 'canon poudre', 'feuillard', 'echalotte', 'meture', 'tuilles', 'ardoises', 'broue terre verrerie', 'grements navire', 'meule moulin', 'moulin verge', 'frais moules peche poisson', 'vesces', 'etoupe', 'marchandises negriere traite', 'casse verre', 'turbe', 'bois copeaux pieces']


### Export des matchs évidents dans un fichier csv

In [6]:
# écriture des matchs au format csv

with open('dumps/product_matching/easy_match_navigo_toflitsimplification.csv', 'w', newline='') as csvfile:
        fieldnames = ['matching_stop_words_cleaned_name', 'navigo_product_id', 'name_navigo', 'product_toflitsimplification']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in transfo_names_products_navigo:
            for j in transfo_names_products_toflit_simplification:
                
                if i['stop_words_cleaned_name'] == j['stop_words_cleaned_name']: # it's a match !
                    writer.writerow({'matching_stop_words_cleaned_name' : i['stop_words_cleaned_name'], 'navigo_product_id': i['navigo_product_id'], 'name_navigo': i['name_navigo'], 'product_toflitsimplification': j['name_toflit_simplification']})

with open('dumps/product_matching/easy_match_navigo_toflitorthographic.csv', 'w', newline='') as csvfile:
        fieldnames = ['matching_stop_words_cleaned_name', 'navigo_product_id', 'name_navigo', 'product_toflitorthographic']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in transfo_names_products_navigo:
            for j in transfo_names_products_toflit_orthographic:
                
                if i['stop_words_cleaned_name'] == j['stop_words_cleaned_name']: # it's a match !
                    writer.writerow({'matching_stop_words_cleaned_name' : i['stop_words_cleaned_name'], 'navigo_product_id': i['navigo_product_id'], 'name_navigo': i['name_navigo'], 'product_toflitorthographic': j['name_toflit_orthographic']})

## 4. racinisation (stemming) : pour les produits qui n'ont toujours pas trouvé de match, on remonte à leur radical

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("french") # choix du language

transfo_names_products_navigo_for_ortho = transfo_names_products_navigo # copie car on a besoin de 2 structures qui évoluent indépendamment en fonction de si on veut fonctionner avec orthographic ou simplification à partir d'ici

for i in transfo_names_products_navigo:
    if i['stop_words_cleaned_name'] in a_aligner_simplification: # on racinise les noms des produits restant à aligner (on choisit simplification car c'est ce qui nous donne le plus de produits à aligner, on restreindra pour orthographic)
        k=0
        stemmed_string=""
        for token in word_tokenize(i['stop_words_cleaned_name']): # chaque mot est racinisé (un token par mot)
            if (k != 0): # je rajoute des espaces à la main que j'avais perdu avec la tokenisation, il doit y avoir mieux comme manière de faire
                stemmed_string = stemmed_string + " " 
            k += 1
            stemmed_string += stemmer.stem(token)
        i['stemmed_name'] = stemmed_string
"""
print(" -------------------- 10 objets navigo a aligner avec toflit simplification :")
for i in transfo_names_products_navigo[0:10]:
    if 'stemmed_name' in i :
        print (i)
"""

for i in transfo_names_products_navigo_for_ortho:
    if i['stop_words_cleaned_name'] in a_aligner_simplification: # on racinise les noms des produits restant à aligner (on choisit simplification car c'est ce qui nous donne le plus de produits à aligner, on restreindra pour orthographic)
        k=0
        stemmed_string=""
        for token in word_tokenize(i['stop_words_cleaned_name']): # chaque mot est racinisé (un token par mot)
            if (k != 0): # je rajoute des espaces à la main que j'avais perdu avec la tokenisation, il doit y avoir mieux comme manière de faire
                stemmed_string = stemmed_string + " " 
            k += 1
            stemmed_string += stemmer.stem(token)
        i['stemmed_name'] = stemmed_string
"""
print(" -------------------- 10 objets navigo a aligner avec toflit orthographic :")
for i in transfo_names_products_navigo_for_ortho[0:10]:
    if 'stemmed_name' in i :
        print (i)
"""

for i in transfo_names_products_toflit_simplification:
        k=0
        stemmed_string=""
        for token in word_tokenize(i['stop_words_cleaned_name']):
            if (k != 0): 
                stemmed_string = stemmed_string + " " 
            k += 1
            stemmed_string += stemmer.stem(token)
        i['stemmed_name'] = stemmed_string
# print(" -------------------- 10 toflit simplification objects:\n", transfo_names_products_toflit_simplification[0:10]) 

for i in transfo_names_products_toflit_orthographic:
        k=0
        stemmed_string=""
        for token in word_tokenize(i['stop_words_cleaned_name']):
            if (k != 0): 
                stemmed_string = stemmed_string + " " 
            k += 1
            stemmed_string += stemmer.stem(token)
        i['stemmed_name'] = stemmed_string
# print(" -------------------- 10 toflit orthographic objects:\n", transfo_names_products_toflit_orthographic[0:10]) 

## 5. fonction de similarité jaccard pour proposer des matchs moins évidents 
**choisi comme la solution la plus pertinente ; expérimentalement, seuil d'acceptation fixé à 1 (on veut restreindre au max les faux positifs, mais on en a encore de trop)**

In [8]:
from fog.metrics import jaccard_similarity

#1 création de listes qui aggrègent les noms navigos restant à aligner et les propositions de matching avec jaccard
proposal_jaccard_simplification = [] 
proposal_jaccard_simplification_to_csv = [] # nous servira au moment d'exporter les resultats en csv
proposal_jaccard_orthographic = [] 
proposal_jaccard_orthographic_to_csv = []


#2 initialisations des listes avec les produits navigo à matcher
for i in transfo_names_products_navigo:
    if 'stemmed_name' in i : # selection des produits navigo qu'il reste à aligner avec toflit simplification
        proposal_jaccard_simplification.append([i]) # on crée une liste de listes qui commencent toujours par l'objet navigo pour lequel on recherche un match
        proposal_jaccard_simplification_to_csv.append([i['navigo_product_id'], i['name_navigo']])
# print(proposal_jaccard_simplification)

for i in transfo_names_products_navigo_for_ortho:
    if 'stemmed_name' in i : # selection des produits navigo qu'il reste à aligner avec toflit orthographic
        proposal_jaccard_orthographic.append([i]) 
        proposal_jaccard_orthographic_to_csv.append([i['navigo_product_id'], i['name_navigo']])
# print(proposal_jaccard_orthographic)


#3 complétion des listes avec les propositions de match
for i in proposal_jaccard_simplification: # i est une liste qui contient un objet navigo à aligner
    for j in transfo_names_products_toflit_simplification: # j est un objet toflit simplification
        if jaccard_similarity(i[0]['stemmed_name'],j['stemmed_name']) == 1:
            # je récupère index de i
            index = proposal_jaccard_simplification.index(i)
            # dans la sous-liste, je mets à la pelle les noms dans toflit simplification qui, cleanés et racinisés sont similaires aux noms navigos
            proposal_jaccard_simplification[index].append(j['name_toflit_simplification'])
            proposal_jaccard_simplification_to_csv[index].append(j['name_toflit_simplification']) 

for i in proposal_jaccard_orthographic: 
    for j in transfo_names_products_toflit_orthographic: 
        if jaccard_similarity(i[0]['stemmed_name'],j['stemmed_name']) == 1:
            index = proposal_jaccard_orthographic.index(i)
            proposal_jaccard_orthographic[index].append(j['name_toflit_orthographic'])
            proposal_jaccard_orthographic_to_csv[index].append(j['name_toflit_orthographic']) 
            
print("************ proposition de matchs avec jaccard (navigo / toflit simplification) ***********")
for i in proposal_jaccard_simplification: # i est une liste
    for j in i: # j est un dict, ou un str
        if isinstance(j,dict): 
            print("\nproduit navigo:", j['name_navigo'])
        else:
            print("candidat toflit:", j) 

print("\n\n************ proposition de matchs avec jaccard (navigo / toflit orthographic) ***********")
for i in proposal_jaccard_orthographic: # i est une liste
    for j in i: # j est un dict, ou un str
        if isinstance(j,dict): 
            print("\nproduit navigo:", j['name_navigo'])
        else:
            print("candidat toflit:", j) 

************ proposition de matchs avec jaccard (navigo / toflit simplification) ***********

produit navigo: Diverses marchandises

produit navigo: Soude
candidat toflit: soudes

produit navigo: Biscuits
candidat toflit: biscuit

produit navigo: Voiles
candidat toflit: voile
candidat toflit: olives
candidat toflit: olivier
candidat toflit: viole

produit navigo: Graines de lin
candidat toflit: graine de lin
candidat toflit: grains ails
candidat toflit: grains de lin
candidat toflit: graine et graine de lin

produit navigo: Bois merrain
candidat toflit: bois et merrains

produit navigo: Marchandises d'un naufrage

produit navigo: Pêche des moules

produit navigo: Sardines pressées de Bretagne

produit navigo: Froment

produit navigo: Grain
candidat toflit: graine
candidat toflit: grains
candidat toflit: graine ???
candidat toflit: graine de
candidat toflit: graine de l ???

produit navigo: Résine
candidat toflit: résines

produit navigo: Fourrage
candidat toflit: fourrages

produit nav

### Si on veut exporter les propositions de match permis par la similarité jaccard dans un fichier csv

In [9]:
# écriture des propositions de matchs au format csv (1ere colonne : id navigo, 2e : nom navigo, colonnes suivantes : propositions de match chez toflit trouvées par jaccard)
with open('dumps/product_matching/jaccard_match_proposal_navigo_toflitsimplification.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['navigo_product_id', 'name_navigo', 'proposals_toflitsimplification'])
            writer.writerows(proposal_jaccard_simplification_to_csv)

with open('dumps/product_matching/jaccard_match_proposal_navigo_toflitorthographic.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['navigo_product_id', 'name_navigo', 'proposals_toflitorthographic'])
            writer.writerows(proposal_jaccard_orthographic_to_csv)

### Liens vers mes résultats

**Alignement navigo / toflit orthographic :**
- easy matching https://docs.google.com/spreadsheets/d/1RLVLy6M23HWRZ5ol88V1wQaDXQEIgePZRRXlk5825v4/edit?usp=sharing
- propositions de matching avec jaccard https://docs.google.com/spreadsheets/d/1b38GkQgNE3wn4pW0XjeT2c3Iuy5asgawHZVtcwq9HNQ/edit?usp=sharing

**Alignement navigo / toflit simplification :**
- easy matching : https://docs.google.com/spreadsheets/d/1rSh9MUqYS2SVURNc0M29m2Ml2TXksqwIY7ZVG5Gfs2o/edit?usp=sharing
- propositions de matching avec jaccard : https://docs.google.com/spreadsheets/d/18MNbLV0zNcE26LbVPsbCF6r_avi_tFpCnaLIEB7iCMg/edit?usp=sharing 

### Tests infructueux avec d'autres fonctions de similarité

In [50]:
from fog.metrics import dice_coefficient, overlap_coefficient
from nltk import edit_distance

# pas de résultats satisfaisants avec ces techniques pour l'instant
proposal_dice = []
for i in transfo_names_products_navigo:
    if 'stemmed_name' in i :
        proposal_dice.append([i])
# print(proposal_dice)

proposal_overlap = []
for i in transfo_names_products_navigo:
    if 'stemmed_name' in i :
        proposal_overlap.append([i]) 
# print(proposal_overlap)

proposal_levenshtein = []
for i in transfo_names_products_navigo:
    if 'stemmed_name' in i :
        proposal_levenshtein.append([i]) 
# print(proposal_levenshtein)

for i in proposal_dice:
        if dice_coefficient(i[0]['stemmed_name'],j['stemmed_name']) >= 0.6:
            index = proposal_dice.index(i)
            proposal_dice[index].append(j['name_toflit_simplification'])

for i in proposal_overlap:
        if overlap_coefficient(i[0]['stemmed_name'],j['stemmed_name']) >= 0.6:
            index = proposal_overlap.index(i)
            proposal_overlap[index].append(j['name_toflit_simplification'])
            
for i in proposal_levenshtein:       
        if edit_distance(i[0]['stemmed_name'],j['stemmed_name']) <= 60:
            index = proposal_levenshtein.index(i)
            proposal_levenshtein[index].append(j['name_toflit_simplification'])
            
print("\n\n************ proposal dice ***********")
for i in proposal_dice: 
    for j in i: 
        if isinstance(j,dict): 
            print("\nproduit navigo:", j['name_navigo'])
        else:
            print("candidat toflit:", j) 

print("\n\n************ proposal overlap ***********")
for i in proposal_overlap: 
    for j in i: 
        if isinstance(j,dict): 
            print("\nproduit navigo:", j['name_navigo'])
        else:
            print("candidat toflit:", j) 

print("\n\n************ proposal levenshtein ***********")
for i in proposal_levenshtein: 
    for j in i: 
        if isinstance(j,dict): 
            print("\nproduit navigo:", j['name_navigo'])
        else:
            print("candidat toflit:", j) 

KeyError: 'name_toflit_simplification'